# Analysis of Layout ID

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1nkdkx2rI6nVKgoKBgkCUtfwEwuv8kptrRUXcXtfv0NM/edit?gid=247168398#gid=247168398]
- (Hypothesis for Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1TSTxMCgEvuoayzOfx1MUqlV0tiqsVTBRN8aldlnFXxA/edit?gid=0#gid=0]

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json

In [2]:
# pythpon client auth & login
# init the client
client = BackendPythonClient(
    backend_endpoint="https://dev.green-fusion.de/services/energy-management-backend/v1/api",
)
#login
client.login(
    auth_endpoint="https://auth.green-fusion.de",
    realm="development",
    client_id="backend_python_client", # backend_python_client, datascience_development_api
    grant_type="device_code", #"client_credentials",  # or "password", "device_code"
)

In [4]:
def get_buildings_df(client, customers):
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id = getattr(b, 'building_id', None)
                modular_system_str = getattr(b, 'modular_system', None)
                address = getattr(b, 'address', None)
                
                modular_system = None
                layout_id = None
                
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        # Extract layoutID if exists
                        layout_id = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        modular_system = None
                
                data.append({
                    #"customer_id": customer_id,
                    "customer_name": customer_name,
                    "building_id": building_id,
                    "address": address,
                    "modular_system": modular_system,
                    "layout_id": layout_id
                })

        return pd.DataFrame(data)
    
    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

In [5]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df = get_buildings_df(client, customers)


In [6]:
df.head(10)

,customer_name,building_id,address,modular_system,layout_id
0,Stadtwerke Troisdorf,571,Asselbachstr. 40,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--global-separation-circuit:1--hea...
1,Stadtwerke Troisdorf,574,Lütticher Str. 12,"{'publishedAt': '2024-06-04T07:43:15.376Z', 'n...",gas:1--gas:2--global-separation-circuit:1--hea...
2,Stadtwerke Troisdorf,575,Rathausallee 10,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--global-separation-circuit:1--hea...
3,Stadtwerke Troisdorf,738,Lohmarer Str. 33,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--chp:1--buffer-tank:1--global-sep...
4,Stadtwerke Troisdorf,739,Zum Altenforst 10,"{'name': 'modular-system-entry', 'id': 'modula...",gas:1--gas:2--chp:1--buffer-tank:1--global-sep...
5,LÜBECKER BAUVEREIN,643,Lützowstraße 30-46,None,None
6,LÜBECKER BAUVEREIN,644,Ratzeburger Allee 76-84,None,None
7,LÜBECKER BAUVEREIN,645,Tannenbergstr. 6-12,None,None
8,LÜBECKER BAUVEREIN,646,Andersenring 28,None,None
9,LÜBECKER BAUVEREIN,647,"Brolingstr. 16, Geverdestr. 58, Ludwigstr. 63/65",None,None


In [8]:
df.describe()

,building_id
count,1699.000000
mean,1175.504414
std,675.796589
min,1.000000
25%,560.500000
50%,1319.000000
75%,1773.500000
max,2320.000000


# export df

In [ ]:
if True:
    df.drop(columns=['modular_system'])

# Analyse

In [9]:
df.count()

customer_name     1699
building_id       1699
address           1699
modular_system     439
layout_id          439
dtype: int64